# Ball Drop

In [ ]:
from IPython.display import display, SVG
import numpy as np
import pydot

from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, AngleAxis,
                         Concatenate, DiagramBuilder, InputPortIndex,
                         LeafSystem, MeshcatVisualizer, Parser,
                         PiecewisePolynomial, PiecewisePose, PointCloud, RandomGenerator, RigidTransform,
                         RollPitchYaw, Simulator, StartMeshcat,
                         UniformlyRandomRotationMatrix,
                         BsplineTrajectory, Sphere, Rgba,
                         KinematicTrajectoryOptimization, Solve, MinimumDistanceConstraint,
                         PositionConstraint)

from manipulation.scenarios import (AddFloatingRpyJoint, AddRgbdSensor,
                                    AddShape, ycb)

from scenarios import *
#from controllers import *

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://68a898be-bd4e-4046-916a-6293aadf6c5d.deepnoteproject.com/7006/


In [ ]:
import numpy as np

import pydrake.all
from pydrake.all import *


# Adds a ball to the scene
def AddBall(plant, name='ball'):
    parser = pydrake.multibody.parsing.Parser(plant)
    ball = parser.AddModelFromFile("models/ball.sdf", name)
    return ball




def SetupBall(scenario, context, pos=[0,0,0], vel=[0,0,0]):
    plant = scenario.plant
    ball = scenario.ball
    plant_context = plant.GetMyMutableContextFromRoot(context)

    plant.SetPositions(plant_context, ball, np.hstack((1,0,0,0,pos)))
    plant.SetVelocities(plant_context, ball, np.hstack((0,0,0,vel)))


def SimulateWithMeshcat():
    builder = DiagramBuilder()
    scenario = Scenario()
    station = builder.AddSystem(scenario.get_diagram())
    

    #integrator1 = SetupController(builder, scenario, station, name='iiwa1')
    #integrator2 = SetupController(builder, scenario, station, name='iiwa2')

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("geometry_query"), meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)

    #SetupIntegrator(integrator1, context)
    #SetupIntegrator(integrator2, context)

    # Change these values to change the ball's initial position
    # and velocity
    SetupBall(scenario, context, 
        pos=[2, 0, 1.2], 
        vel=[0.01, 0, 0]
    )

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)

    camera = AddRgbdSensor(builder, scene_graph, RigidTransform(
        RollPitchYaw(np.pi, 0, np.pi / 2.0), [0, 0, .8]))  #add camera with a certain pose to the scene
    builder.ExportOutput(camera.color_image_output_port(), "color_image")

    simulator = Simulator(diagram, context)
    # simulator.set_target_realtime_rate(1.0)
    # simulator.AdvanceTo(0.01)

    return simulator, visualizer

simulator, visualizer = SimulateWithMeshcat()

RuntimeError: DiagramBuilder: Build() or BuildInto() has already been called to create a Diagram; this DiagramBuilder may no longer be used.

In [ ]:
simulator.set_target_realtime_rate(0.5)
meshcat.AddButton("Stop Simulation", "Escape")
print("Press Escape to stop the simulation")
while meshcat.GetButtonClicks("Stop Simulation") < 1:
    simulator.AdvanceTo(simulator.get_context().get_time() + .05)
meshcat.DeleteButton("Stop Simulation")

Press Escape to stop the simulation


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=68a898be-bd4e-4046-916a-6293aadf6c5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>